<a href="https://colab.research.google.com/github/piccigus/-Explorando-Dados/blob/main/Explorando_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício simples 01



In [10]:
def Soma(var1:float,var2:float) -> int:

    """ Esta função retorna o resultado da soma entre dois números.
      var1: variável numérica
      var2: variável numérica
      resultado: resultado da soma de var1 e var2
      """
    resultado = var1 + var2
    print('O resultado da soma é: ', resultado)

    if resultado > 10:
     print('Só Alegria!!')

    return resultado

In [11]:
Soma (1,10)

O resultado da soma é:  11
Só Alegria!!


11

**Configurando o Kernel Spark**

In [12]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [13]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [14]:
from pyspark.sql import SparkSession
 
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

**Explorando os Dados**

In [15]:

# carregar dados do Público de Modelagem
app_train = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/application_train.csv", inferSchema=True, header=True)
 
app_train.registerTempTable('app_train')
 
app_train.count()

307511

In [16]:

# carregar dados da Tabela Bureau
bureau = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/bureau.csv", inferSchema=True, header=True)
 
bureau.registerTempTable('bureau')
 
bureau.count()

1716428

**Verificando variáveis da tabela primcipal**

In [17]:
app_train.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [18]:
 string_list = [item[0] for item in app_train.dtypes if item[1].startswith('string')]
 string_list


['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

**Exercícios sobre a tabela de modelagem**

Calcule a volumetria e taxa de evento para cada domínio de cada uma das 

variáveis tipo string.

In [19]:
spark.sql("""
            select 
              NAME_CONTRACT_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              NAME_CONTRACT_TYPE
            order by 
              AVG_TARGET desc
""").show()


+------------------+----------+------+
|NAME_CONTRACT_TYPE|AVG_TARGET|Volume|
+------------------+----------+------+
|        Cash loans|      8.35|278232|
|   Revolving loans|      5.48| 29279|
+------------------+----------+------+



In [25]:
spark.sql("""
            select 
              CODE_GENDER,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              CODE_GENDER
            order by 
              AVG_TARGET desc
""").show()


+-----------+----------+------+
|CODE_GENDER|AVG_TARGET|Volume|
+-----------+----------+------+
|          M|     10.14|105059|
|          F|       7.0|202448|
|        XNA|       0.0|     4|
+-----------+----------+------+



In [21]:
spark.sql("""
            select 
              FLAG_OWN_CAR,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              FLAG_OWN_CAR
            order by 
              AVG_TARGET desc
""").show()


+------------+----------+------+
|FLAG_OWN_CAR|AVG_TARGET|Volume|
+------------+----------+------+
|           N|       8.5|202924|
|           Y|      7.24|104587|
+------------+----------+------+



In [22]:
spark.sql("""
            select 
              FLAG_OWN_REALTY,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              FLAG_OWN_REALTY
            order by 
              AVG_TARGET desc
""").show()


+---------------+----------+------+
|FLAG_OWN_REALTY|AVG_TARGET|Volume|
+---------------+----------+------+
|              N|      8.32| 94199|
|              Y|      7.96|213312|
+---------------+----------+------+



Analise de um caso onde quem faz empréstimo do tipo rotativo (Revolving loans) possui menor taxa de evento do que quem faz empréstimo do tipo dinheiro (Cash loans).

In [23]:
spark.sql("""
            select 
              OCCUPATION_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              OCCUPATION_TYPE
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
|     OCCUPATION_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|  Low-skill Laborers|     17.15|  2093|
|             Drivers|     11.33| 18603|
|Waiters/barmen staff|     11.28|  1348|
|      Security staff|     10.74|  6721|
|            Laborers|     10.58| 55186|
|       Cooking staff|     10.44|  5946|
|         Sales staff|      9.63| 32102|
|      Cleaning staff|      9.61|  4653|
|       Realty agents|      7.86|   751|
|         Secretaries|      7.05|  1305|
|      Medicine staff|       6.7|  8537|
|Private service s...|       6.6|  2652|
|                null|      6.51| 96391|
|            IT staff|      6.46|   526|
|            HR staff|      6.39|   563|
|          Core staff|       6.3| 27570|
|            Managers|      6.21| 21371|
|High skill tech s...|      6.16| 11380|
|         Accountants|      4.83|  9813|
+--------------------+----------+------+



In [24]:
spark.sql("""
            select 
              NAME_EDUCATION_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume
 
            from 
              app_train
            group by 
              NAME_EDUCATION_TYPE
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
| NAME_EDUCATION_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|     Lower secondary|     10.93|  3816|
|Secondary / secon...|      8.94|218391|
|   Incomplete higher|      8.48| 10277|
|    Higher education|      5.36| 74863|
|     Academic degree|      1.83|   164|
+--------------------+----------+------+



# Verificando a tabela de modelagem

In [26]:
app_train.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+------------

In [27]:
bureau.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

In [28]:
bureau.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)



Selecione um único indivíduo da tabela principal, filtrar este indivíduo na tabela bureau e verificar seu comportamento transacional

O ID (SK_ID_CURR) escolhido foi 100002:

In [29]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              bureau
            where 
            SK_ID_CURR =  100002 
""").show()


        

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    100002|     6158904|       Closed|     currency 1|      -1125|                 0|            -1038.0|          -1038

In [30]:
spark.sql("""
            select 
              CREDIT_ACTIVE,
              count(*) as Volume
                 
            from 
              bureau
            group by
            CREDIT_ACTIVE
            order by 
            Volume desc  
             
""").show()


+-------------+-------+
|CREDIT_ACTIVE| Volume|
+-------------+-------+
|       Closed|1079273|
|       Active| 630607|
|         Sold|   6527|
|     Bad debt|     21|
+-------------+-------+



In [31]:
spark.sql("""
            select 
              CREDIT_CURRENCY,
              count(*) as Volume
                 
            from 
              bureau
            group by
            CREDIT_CURRENCY
            order by 
            Volume desc  
             
""").show()

+---------------+-------+
|CREDIT_CURRENCY| Volume|
+---------------+-------+
|     currency 1|1715020|
|     currency 2|   1224|
|     currency 3|    174|
|     currency 4|     10|
+---------------+-------+



In [32]:
spark.sql("""
            select 
              DAYS_CREDIT,
              count(*) as Volume
                 
            from 
              bureau
            group by
            DAYS_CREDIT
            order by 
            Volume desc  
             
""").show()

+-----------+------+
|DAYS_CREDIT|Volume|
+-----------+------+
|       -364|  1330|
|       -336|  1248|
|       -273|  1238|
|       -357|  1218|
|       -343|  1203|
|       -315|  1202|
|       -371|  1196|
|       -365|  1194|
|       -210|  1193|
|       -245|  1192|
|       -329|  1190|
|       -420|  1189|
|       -259|  1189|
|       -182|  1180|
|       -175|  1174|
|       -363|  1162|
|       -301|  1160|
|       -392|  1157|
|       -308|  1154|
|       -196|  1152|
+-----------+------+
only showing top 20 rows



# Indicadores Preditivos (Variáveis Explicativas)


In [33]:
# carregar dados do tabela metadados
metadados = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/HomeCredit_columns_description.csv", inferSchema=True, header=True)
 
metadados.registerTempTable('metadados')
 
metadados.count()

219

In [34]:
spark.sql("""
            select 
                 *
            from 
              metadados
            where Table = 'bureau.csv' 
""").show(100,False)

+---+----------+----------------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|_c0|Table     |Row                   |Description                                                                                                        |Special                              |
+---+----------+----------------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|125|bureau.csv|SK_ID_CURR            |ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau |hashed                               |
|126|bureau.csv|SK_BUREAU_ID          |Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application)          |hashed                               |
|127|bureau.csv|CREDIT_ACTIVE 

#Criar uma nova tabela spark

In [35]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(avg(DAYS_CREDIT),2) as AVG_DAYS_CREDIT
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [36]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()



+----------+---------------+
|SK_ID_CURR|AVG_DAYS_CREDIT|
+----------+---------------+
|    100002|         -874.0|
+----------+---------------+



Exercício: crie variáveis preditivas, aplicando as estatísticas min, max e avg, para todas as variáveis numéricas contidas na tabela Bureau

# Mínimo

In [48]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(min(DAYS_CREDIT_ENDDATE),2) as MIN_DAYS_CREDIT_ENDDATE 
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [57]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(min(DAYS_CREDIT),2) as MIN_DAYS_CREDIT
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [56]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(min(DAYS_ENDDATE_FACT),2) as MIN_DAYS_ENDDATE_FACT     
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [51]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(min(DAYS_CREDIT_UPDATE),2) as MIN_DAYS_CREDIT_UPDATE        
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [55]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()


+----------+----------------------+
|SK_ID_CURR|MIN_DAYS_CREDIT_UPDATE|
+----------+----------------------+
|    100002|                 -1185|
+----------+----------------------+



# Máximo


In [58]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(max(DAYS_CREDIT_UPDATE),2) as Max_DAYS_CREDIT_UPDATE        
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [59]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()


+----------+----------------------+
|SK_ID_CURR|Max_DAYS_CREDIT_UPDATE|
+----------+----------------------+
|    100002|                    -7|
+----------+----------------------+



In [60]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(max(DAYS_CREDIT),2) as Max_DAYS_CREDIT                   
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [61]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+---------------+
|SK_ID_CURR|Max_DAYS_CREDIT|
+----------+---------------+
|    100002|           -103|
+----------+---------------+



In [64]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(max(DAYS_CREDIT_ENDDATE),2) as Max_DAYS_CREDIT_ENDDATE                          
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [65]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+-----------------------+
|SK_ID_CURR|Max_DAYS_CREDIT_ENDDATE|
+----------+-----------------------+
|    100002|                  780.0|
+----------+-----------------------+



# Média

In [66]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(avg(DAYS_CREDIT),2) as AVG_DAYS_CREDIT
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [67]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+---------------+
|SK_ID_CURR|AVG_DAYS_CREDIT|
+----------+---------------+
|    100002|         -874.0|
+----------+---------------+



In [68]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(avg(DAYS_CREDIT_ENDDATE),2) as AVG_DAYS_CREDIT_ENDDATE   
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [69]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+-----------------------+
|SK_ID_CURR|AVG_DAYS_CREDIT_ENDDATE|
+----------+-----------------------+
|    100002|                 -349.0|
+----------+-----------------------+



In [70]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(avg(DAYS_ENDDATE_FACT),2) as AVG_DAYS_ENDDATE_FACT        
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [71]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+---------------------+
|SK_ID_CURR|AVG_DAYS_ENDDATE_FACT|
+----------+---------------------+
|    100002|               -697.5|
+----------+---------------------+



In [72]:
df_bureau_01 = spark.sql("""
            select 
                 SK_ID_CURR,
                 round(avg(DAYS_CREDIT_UPDATE),2) as AVG_DAYS_CREDIT_UPDATE            
            from 
              bureau
            group by 
              SK_ID_CURR
""")
 
df_bureau_01.registerTempTable('df_bureau_01')
 
df_bureau_01.count()

305811

In [73]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()

+----------+----------------------+
|SK_ID_CURR|AVG_DAYS_CREDIT_UPDATE|
+----------+----------------------+
|    100002|               -499.88|
+----------+----------------------+



# Indicadores Preditivos considerando regras de negócio.



In [ ]:
def_bureau_01 = spark.sql( """
             select
                 SK_ID_CURR,
                 round(avg(abs(DAYS_CREDIT)), 2) as AVG_DAYS_CREDIT,
                 round(avg(case when CREDIT_ACTIVE in ('Active') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CRED_ACTV_ACTIVE,
                 round(avg(case when CREDIT_ACTIVE in ('Closed') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CRED_ACTV_CLOSE,
                 round(avg(case when CREDIT_ACTIVE in ('Sold') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CRED_ACTV_SOLD,
                 round(avg(case when CREDIT_ACTIVE in ('Bad debt') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CRED_ACTV_BADDBT,

                 round(sum(abs(DAYS_CREDIT)),2 as SUM_DAYS_CREDIT,
                 round(sum(case when CREDIT_ACTIVE in ('Active')then abs(DAYS_CREDIT)else 0 end),2) as SUM_CRED_ACTV_ACTIVE,
                 round(sum(case when CREDIT_ACTIVE in ('Closed')then abs(DAYS_CREDIT)else 0 end),2) as SUM_CRED_ACTV_CLOSE,
                 round(sum(case when CREDIT_ACTIVE in ('Sold')then abs(DAYS_CREDIT)else 0 end),2) as SUM_CRED__ACTV_SOLD,
                 round(sum(case when CREDIT_ACTIVE in ('Bad debt')then abs(DAYS_CREDIT)else 0 end),2) as SUM_CRED_ACTV_BADDBT

            from 
              bureau
            group by 
            SK_ID_CURR 

  """)

def_bureau_01.registerTempTable('def_bureau_01')

def_bureau_01.count()



In [ ]:
def_bureau_01.show()

In [ ]:
### vamos filtrar o indivíduo 100002
spark.sql("""
            select 
                 *
            from 
              df_bureau_01
            where 
            SK_ID_CURR =  100002 
""").show()


# Compondo ABT considerando os indicadores preditivos da tabela Bureau.



In [76]:
 features_expl_list = df_bureau_01.columns
 for feat in features_expl_list:
   var = 'b.' + feat + ','
   print(var)
   

b.SK_ID_CURR,
b.AVG_DAYS_CREDIT_UPDATE,


In [ ]:
abt_bureau = spark.sql("""
          Select 
 
          a.*,
          b.AVG_DAYS_CREDIT,
          b.AVG_CRED_ACTV_ACTIVE,
          b.AVG_CRED_ACTV_CLOSED,
          b.AVG_CRED_ACTV_SOLD,
          b.AVG_CRED_ACTV_BADDBT,
          b.SUM_DAYS_CREDIT,
          b.SUM_CRED_ACTV_ACTIVE,
          b.SUM_CRED_ACTV_CLOSED,
          b.SUM_CRED_ACTV_SOLD,
          b.SUM_CRED_ACTV_BADDBT,
          b.MIN_DAYS_CREDIT,
          b.MIN_CRED_ACTV_ACTIVE,
          b.MIN_CRED_ACTV_CLOSED,
          b.MIN_CRED_ACTV_SOLD,
          b.MIN_CRED_ACTV_BADDBT
 
          from app_train  as a
          left join 
          df_bureau_01 as b
          on
          a.SK_ID_CURR = b.SK_ID_CURR
 
 """)
 
abt_bureau.registerTempTable('abt_bureau')
 
abt_bureau.count()


Agora vamos salvar a tabela gerada no diretório das **ABTs** **negrito**

In [ ]:
abt_bureau.write.mode('overwrite').parquet("/content/drive/MyDrive/SQLMMs/ABT/VARS_BUREAU")

In [ ]:
read_abt_bureau = spark.read.format("parquet").load("/content/drive/MyDrive/SQLMMs/ABT/VARS_BUREAU")
 
read_abt_bureau.count()